In [13]:
!pip install -q -U google-genai

## Entity Extraction into JSON/Lists

In [1]:
from google import genai
# import google.generativeai as genai
from pydantic import BaseModel
import os

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [2]:
from pydantic import BaseModel

class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]

Recipe(recipe_name="Pasta", ingredients=["flour", "water"])
# Recipe(recipe_name=1, ingredients=["flour", "water"])

Recipe(recipe_name='Pasta', ingredients=['flour', 'water'])

In [ ]:
text = f"""Probe ordered into shocking building collapse
PUBLISHED : 30 Mar 2025 at 05:46

NEWSPAPER SECTION: News

WRITER: Aekarach Sattaburuth and Wassayos Ngamkham
0:00 / 1:44
Search and rescue workers and equipment are deployed at the collapse site at the new compound of the State Audit Office in Chatuchak district, Bangkok, on Saturday. Pornprom Satrabhaya
Search and rescue workers and equipment are deployed at the collapse site at the new compound of the State Audit Office in Chatuchak district, Bangkok, on Saturday. Pornprom Satrabhaya
The Department of Public Works and Town and Country Planning (DPT) has been ordered to conduct a thorough investigation into the collapse of the State Audit Office's building during the earthquake.

Prime Minister Paetongtarn Shinawatra said in a media interview that a committee will establish the cause of the structural failure and report back within a week.

The inquiry will examine the building's design, the authority that approved the design, how it was approved and whether any of these factors led to the collapse, she said.

ADVERTISEMENT

Ms Paetongtarn also echoed public concern as to why this building collapsed but other, similar structures remained intact.

RELATED
Are women allowed their own dreams, wonders Chimamanda Ngozi Adichie
life
Are women allowed their own dreams, wonders Chimamanda Ngozi Adichie
South Korea's 'heartbreaking' wildfires expose super-aged society
world
South Korea's 'heartbreaking' wildfires expose super-aged society
"I watched multiple clips of the building collapse from different angles. From my experience in the construction industry, I have never seen an issue like this. We must investigate thoroughly because a significant portion of the budget was allocated, and the deadline for completion had been extended," she said.

The office was a joint venture between Italian-Thai Development Plc and a subsidiary of China Railway No.10 Engineering Group, which operates under the state-owned China Railway Engineering Corporation (CREC) -- one of the world's largest construction and engineering companies.

Netizens were quick to point out that the company's posts about the collapsed building have all been deleted, despite the structure having been completed on March 31 last year.

Meanwhile, Interior Minister Anutin Charnvirakul said rescuers are being hampered by debris and insisted that their operations are not slow.

He did, however, concede that the 24-hour mark, known for being the point where the odds of finding survivors after a disaster drops sharply, has passed.



Please credit and share this article with others using this link: https://www.bangkokpost.com/thailand/general/2990661/probe-ordered-into-shocking-building-collapse. View our policies at http://goo.gl/9HgTd and http://goo.gl/ou6Ip. © Bangkok Post PCL. All rights reserved."""

class NewsExtractor(BaseModel):
    who: str
    what: str
    where: str
    when: str
    why: str

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=f'Summarize news into who, what, where, when and why TEXT:{text}',
    config={
        'response_mime_type': 'application/json',
        'response_schema': NewsExtractor,
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: NewsExtractor = response.parsed
results_store = response.parsed

{
  "who": "Department of Public Works and Town and Country Planning (DPT), Prime Minister Paetongtarn Shinawatra, Interior Minister Anutin Charnvirakul, Italian-Thai Development Plc, China Railway No.10 Engineering Group, Netizens",
  "what": "The Department of Public Works and Town and Country Planning (DPT) has been ordered to conduct a thorough investigation into the collapse of the State Audit Office's building during the earthquake. A committee will establish the cause of the structural failure and report back within a week. Rescuers are hampered by debris.",
  "where": "Chatuchak district, Bangkok, at the new compound of the State Audit Office",
  "when": "Published 30 Mar 2025, collapse occurred sometime before that. Building completed March 31 last year.",
  "why": "The inquiry will examine the building's design, the authority that approved the design, how it was approved and whether any of these factors led to the collapse. Public concern as to why this building collapsed but

In [5]:
# Exercise extract the list and run for loop through it.

In [19]:
class Thaifood(BaseModel):
    Location: str
    food: list[str]

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular food in Thailand based on location.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Thaifood],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Thaifood] = response.parsed

[
  {
    "Location": "Bangkok",
    "food": ["Pad Thai", "Mango Sticky Rice", "Tom Yum Goong", "Boat Noodles"]
  },
  {
    "Location": "Chiang Mai",
    "food": ["Khao Soi", "Sai Oua (Northern Thai Sausage)", "Gaeng Hang Lay (Burmese-style Pork Curry)"]
  },
  {
    "Location": "Southern Thailand (e.g., Phuket, Krabi)",
    "food": ["Massaman Curry", "Roti", "Seafood (Grilled Fish, Tom Yum Seafood)"]
  },
  {
    "Location": "Ayutthaya",
    "food": ["Roti Sai Mai", "River Prawns"]
  }
]


In [7]:
# Exercise extract the list and run for loop through it.
# Exercise ask the model list cities in Thailand, make model return a list of city. Run For loop through it.

In [8]:
class ThaifoodBangkok(BaseModel):
    food: list[str]

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular food in Bangkok.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': ThaifoodBangkok,
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

{
  "food": ["Pad Thai", "Mango Sticky Rice", "Tom Yum Soup", "Green Curry", "Som Tum (Papaya Salad)", "Boat Noodles"]
}


In [9]:
for food in response.parsed.food:
    print(food)

Pad Thai
Mango Sticky Rice
Tom Yum Soup
Green Curry
Som Tum (Papaya Salad)
Boat Noodles


## Classification

In [10]:
import enum

class Instrument(enum.Enum):
  PERCUSSION = "Percussion"
  STRING = "String"
  WOODWIND = "Woodwind"
  BRASS = "Brass"
  KEYBOARD = "Keyboard"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='What type of instrument is an oboe?',
    config={
        'response_mime_type': 'text/x.enum',
        'response_schema': Instrument,
    },
)

print(response.text)

Woodwind


In [11]:
class Grade(enum.Enum):
    A_PLUS = "a+"
    A = "a"
    B = "b"
    C = "c"
    D = "d"
    F = "f"

class Recipe(BaseModel):
  recipe_name: str
  rating: Grade

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List 10 home-baked cookies and give them grades based on tastiness.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
    },
)

print(response.text)

[
  {
    "recipe_name": "Chocolate Chip Cookies",
    "rating": "a+"
  },
  {
    "recipe_name": "Peanut Butter Cookies",
    "rating": "a"
  },
  {
    "recipe_name": "Oatmeal Raisin Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Sugar Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Snickerdoodle Cookies",
    "rating": "a"
  },
  {
    "recipe_name": "Gingerbread Cookies",
    "rating": "c"
  },
  {
    "recipe_name": "Shortbread Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Macadamia Nut Cookies",
    "rating": "a"
  },
  {
    "recipe_name": "Biscotti Cookies",
    "rating": "c"
  },
  {
    "recipe_name": "White Chocolate Chip Cookies",
    "rating": "b"
  }
]


In [12]:
# Get a newspaper extract date, summary, title in Json format each with strings.